# Machine Learning for text classification

Lets set the stage! After a year full of COVID disruptions the university has made all exams online. Celebrations all around! But wait, what about the cheating you say? Fortunately (or unfortunately), the uni has predicted this, and hired a crack team of Designated Exam Validators (or DEVs for short) to check for plagarism. And you've just been hired! And of course, you're going to use ML to do all your work for you!

# Problem Statement

Given a sentence, we need to find the probability a sentence came from each possible source.

For example, given the text "The University of Auckland began as a constituent college of the University of New Zealand, founded on 23 May 1883 as Auckland University College" the probability for "Wikipedia" = 0.9 and "Twitter" = 0.1

We need to build a machine learning classifier to do this for us!

# 1) Collect data

First, we need some data from each source. This will allow us to find patterns, i.e., learn what makes *wikipedia* sentences different to *twitter* sentences

This is normally the *most important part of ML*, but we've taken care of it! In the github repo there is a data folder containing some datasets from different sources.

First we will just load this in.

In [ ]:
# Load in some libraries
# The pathlib library makes handling filepaths easier, letting us open data files.
import pathlib
import pandas as pd
import random

# Set the folder containing our data
data_path = pathlib.Path('data')

# Setup availiable filenames
chess_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/chess.txt"
music_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/music.txt"
happy_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/happy_topical_chat.txt"
trumpspeech_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/trumpSpeech.txt"
wallstreetbets_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/wallstreetbets_comments.txt"
javascript_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/javascript.txt"
shakespeare_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/shakespeare.txt"
wiki_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/wiki.txt"
twitter_filename = "https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/twitter2.txt"

# Just a helper function, don't worry about this one!
def get_file_or_cache(path):
    cache = None
    def get():
        nonlocal cache
        if cache is None:
            # with open(path, 'r', encoding='utf8') as f:
            #     cache = f.readlines()
            print(path)
            df = pd.read_csv(path, delimiter = "\n")
            cache = list([str(x[0]) for x in df.values])
        return cache
    return (get, path.split('/')[-1].split('.')[0])

# Construct a list of possible files and their names
data_files = [get_file_or_cache(x) for x in [chess_filename, music_filename, happy_filename, trumpspeech_filename, javascript_filename, shakespeare_filename, wiki_filename]]

# Construct dataset of sentences and labels from every source
# Note! Rather than using the actual names for the datasets, we give each one
# its own numeric ID.
# We record these so we can swap between human readable name and ID.
X = []
Y = []
source_IDs = {}
ID_to_source = {}
source_ID = 0
for source_constructor, source in data_files:
    source_IDs[source] = source_ID
    ID_to_source[source_ID] = source
    lines = source_constructor()
    for line in lines:
        X.append(line)
        Y.append(source_ID)
    source_ID += 1

observations = list(zip(X, Y))

https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/chess.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/music.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/happy_topical_chat.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/trumpSpeech.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/javascript.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/shakespeare.txt
https://raw.githubusercontent.com/BenHals/DevsMLWorkshop/main/data/wiki.txt


Now we have our data loaded into X, a list of sentences, and Y, a list of the sources each sentence came from.
Lets check out some sample lines from each source!

In [ ]:
inspect_index = int(random.random() * len(observations))
for index, (line, source) in enumerate(observations[inspect_index:inspect_index+5]):
    print("Observation", index)
    print("X{} = ".format(index), line)
    # Source is stored as a number, representing the source (explained later!)
    print("Y{} =".format(index), source, "Source name:", ID_to_source[source])
    print('-----------')

# How do we *actually* predict?

The goal is to learn a model which can tell the difference between classes. For example, consider sentences from Wikipedia (W) or Twitter (T). If we show our model a new input sentence (X) where we do NOT know the origin, our model should be able to tell where the sentence came from. 

Formally, the $i^{th}$ input is called $x_{i}$ and its true class is called $y_{i}$.
For us, $x_{i}$ is the $i^{th}$ sentence we need to label and $y_{i}$ is where this sentence actually came from.
Our model predicts some label, $\hat y_{i}$, for this sentence (so *wikipedia* or *twitter*).
We want to train it so that *our* label, $\hat y_{n}$, is close to the *true* label, $y_{i}$.

There are many, many different ways to do this prediction, and we will look at a simple one called Naive Bayes.

Lets true a manual version to get the idea:


In [ ]:
# Pick two random labels to choose from
g, label_opt1 = random.choice(data_files)
g, label_opt2 = random.choice(data_files)

# Select a random observations from a random source
source_gen, true_label = random.choice(data_files)

# Shuffle labels
label_options = [label_opt1, label_opt2, true_label]
random.shuffle(label_options)

lines = source_gen()
line = random.choice(lines)
print(line)
predicted_label = input("Where did this sentence come from? {} or {} or {}?\n".format(*label_options))

#****** YOUR CODE HERE! ***********
# First challenge to get you started:
# How can we tell if the predicted label (what the user typed for now)
# matches the true label?
prediction_iscorrect = predicted_label == true_label
#***********************************

# Now we use whether or not the prediction was correct to inform the user.
print("You were", "Right" if prediction_iscorrect else "Wrong", "!")
print("Your predicted label (y^) was: {}. The true label (y) was {}".format(predicted_label, true_label))

## Using a model to make a prediction
 
 How can we *train the computer* to predict a class for us? 
 Lets think of a simple example: We are given a sentence ($x$), and need to predict the most likely class ($\hat y$), Wikipedia (W) or Twitter (T). We start by asking a friend what class they think is most likely and they say Wikipedia has a 70% chance and Twitter and 30%. 

 In mathmatical terms we can rewrite the "_probability that the class is Wikipedia given this specific sentence $x$ , $\hat y = W$, is 0.7_" as $p(\hat y = W | x) = 0.7$. Similarly, we can write $p(\hat y = T | x) = 0.3$ for the sentence being from twitter.

 Now it is pretty easy to make a prediction, Wikipedia has a 70% chance and Twitter only has a 30% chance so we should predict Wikipedia! 

 ### But how did our friend come up with $p(\hat y| x)$ in the first place?
 This is basically what the computer needs to do, find the probability of a sentence coming from each source.
 This is what Naive Bayes solves!

 ## Naive Bayes - Probability theory (spooky)

 Note: Naive Bayes is pretty simple, and is quite intuitive when you wrap your head around it, but if this is your first introduction to probability it can be quite confusing! If you don't understand at first don't get discoraged! I find that drawing diagrams and thinking about it from a few directions helps really understand.

 The end goal of a model is to calculate $p(y = C|X=x)$. In plain english, this can be read as calculate the _probability that the true class of the input is C given what we know about the sentence_. For a concrete example, lets use the sentence "The University of Auckland was founded on 23 May 1883". We want to predict the probability that $y = Wikipedia$ or $y = Twitter$ given that the sentence $x$ = "The University of Auckland was founded on 23 May 1883". This can be difficult to calculate, and isn't really how we think about things as humans.

 Instead of calculating this directly, _Bayes Theorum_ gives us a way to swap things around.

 $$ p(y=C|X=x) = p(X=x|y=C)p(y=C)$$

 This is like thinking "what is the probability that the sentence was found on twitter (or wikipedia)". This is a bit more natural to think about, and it turns out easier to calculate.
 
 ## Intuition
 Instead of directly trying to work out the probability of wikipedia or twitter, lets look at indivdual words. If we see the words "follow me!" we can say twitter has a pretty high probability. On the other hand, if we see the words "Auckland[1][2]" we could say wikipedia has a high probability. How did we do this?
 
 We looked at the probability of each word (or few words) coming from each source! We know that 'following' is something people on twitter do, so if we see this word we know it is more likely to have come from twitter than wikipedia. This is the $p(x=X|y=C)$ in the formula!
 
Lets see if we can use this to do some machine learning!

# Exercise 1: Calculate word probabilities

We need to *learn* 2 things from our data.
1. What is the probability of seeing each word overall?
    1. This is $p(x=X)$, or the *probability* that a word is X
    2. e.g p('and') is high, it occurs a lot! But p('founded') is low, it doesn't occur too often.
2. What is the probability of seeing each word *from each source*?
    1. This is $p(x=X|y=Y)$, or the *probability* that a word is X if we *know* it is from e.g wikipedia
    2. e.g. p('founded'|Twitter) might be pretty low, its an uncommon word on twitter. But p('founded'|Wikipedia) is high, it occurs all the time on wikipedia!

## Coding Challenge 1 - Calculate the probability of words

Lets start simple - How can we calculate the probability of a word occuring?

One way to think about this: We count combine all our training data into a huge list of words. Lets say we have $N$ words overall (including all repetitions). Then I tell you a word, e.g., "hello". If you closed your eyes and pointed to a random word in the list, what are the chances you would end up pointing to the word "hello"?

We can calulate this as: The number of times a word occurs in the training data, divided by the total number of words.

### First lets count each word

In [ ]:
  def get_word_counts(training_sentences):
    # fill word_counts so it contains the number of times each word occurs.
    # word_counts = {
    #       'and': 700,
    #       'founded': 15,
    #       ....
    #}
    word_counts = {}
    total_num_words = 0

    # We want to look though all the training sentences,
    for sentence in training_sentences:

      # We can use .split to split each sentence into words
      words = sentence.split()

      for word in words:
        # Increment the total
        total_num_words += 1

        # Add word to dictionary if not already in it
        if word not in word_counts:
          word_counts[word] = 0
        
        # Add to the count for the word
        word_counts[word] += 1
    return word_counts, total_num_words

word_counts, total_num_words = get_word_counts(X)
print(f"There are {total_num_words} overall")
print(f"We saw the word 'hello' {word_counts['hello']} times!")
print(f"We saw the word 'and' {word_counts['and']} times!")

Now lets calculate the probabilities:

In [ ]:
def get_probabilities_from_counts(word_counts, total_num_words):
  # fill word probabilities so it contains the probability of each word
  # word_probabilities = {
  #       'and': 0.75,
  #       'founded': 0.01,
  #       ....
  #}
  word_probabilities = {}

  for w in word_counts:
    # Get the count for the word
    w_count = word_counts[w]

    # Calc probability, and save to the dictionary
    word_probabilities[w] = w_count / total_num_words
  return word_probabilities

word_counts, total_num_words = get_word_counts(X)
word_probabilities = get_probabilities_from_counts(word_counts, total_num_words)

print(f"We saw the word 'hello' {word_probabilities['hello'] * 100}% of the time!")
print(f"We saw the word 'and' {word_probabilities['and'] * 100}% of the time!")

# Coding challenge 2 - Separate probabilities for each class

We have just calculated the probabilities of words *overall*.

What we actually need for Naive Bayes is the probabilities of words in each *source*.

How can we update our code to instead calculate the probabilities per source?

*Hint: What do we calculate if we do the same process on only training sentences from one source?*

In [ ]:
  def calulate_conditionals(X, Y):
    # fill word_probs_by_source so it contains a dict for each source
    # each containing each word and probability only in that source
    # word_probs_by_source = {
    #       'Twitter': {
    #              'founded': 0.001,
    #               ...,
    #        },
    #       'Wikipedia': {
    #              'founded': 0.05,
    #              ...,
    #       }
    #}
    
    # Get a list of the possible sources
    # (Note: this is an ID not a name)
    sources = set(Y)
    word_probs_by_source = {}

    for s in sources:
      # Selects all the training examples which come from this source
      source_training_data = [x for x,y in zip(X,Y) if y == s]

      # Extract the counts and total for the specific source
      source_counts, source_total = get_word_counts(source_training_data)

      # Transform into probabilities
      source_probabilities = get_probabilities_from_counts(source_counts, source_total)

      #Save to the dictionary
      word_probs_by_source[s] = source_probabilities
    return word_probs_by_source

word_probs_by_source = calulate_conditionals(X, Y)
print(f"The probability of seeing the word 'wall' in Trumps speechs was {word_probs_by_source[source_IDs['trumpSpeech']]['wall']}")
print(f"The probability of seeing the word 'thou' in shakespeare was {word_probs_by_source[source_IDs['shakespeare']]['thou']}")

In [ ]:
# This will raise an error, because we did not see the word wall!
print(f"The probability of seeing the word 'wall' in chess was {word_probs_by_source[source_IDs['chess']]['wall']}")

### Problem - what about words which don't appear in some sources?

For example, what if we never see the word "Twitter" in any sentence collected from wikipedia?

We will get a probability p("Twitter"|Wikipedia) = 0!

This may be true in our training data, but means that we cannot predict any sentences with Twitter in them...

Solution: Add 1 to the count of *every* word! This avoids all 0s! But we also have to add to the denominator to balance this out.

In [ ]:
  def calulate_conditionals(X, Y, unique_words):
    # fill word_probs_by_source so it contains a dict for each source
    # each containing each word and probability only in that source
    # word_probs_by_source = {
    #       'Twitter': {
    #              'founded': 0.001,
    #               ...,
    #        },
    #       'Wikipedia': {
    #              'founded': 0.05,
    #              ...,
    #       }
    #}
    
    # Get a list of the possible sources
    # (Note: this is an ID not a name)
    sources = set(Y)
    word_probs_by_source = {}

    for s in sources:
      # Selects all the training examples which come from this source
      source_training_data = [x for x,y in zip(X,Y) if y == s]
      # Extract the counts and total for the specific source
      source_counts, source_total = get_word_counts(source_training_data)

      # Add 1 to the count of all words
      for w in unique_words:
        if w not in source_counts:
          source_counts[w] = 0
        source_counts[w] += 1
      
      # Add since we added +1 for each word to the top of the fraction,
      # we need to add to the bottom as well. This is + len(unique_words)
      source_total += len(unique_words)

      # Transform into probabilities
      source_probabilities = get_probabilities_from_counts(source_counts, source_total)

      #Save to the dictionary
      word_probs_by_source[s] = source_probabilities
    return word_probs_by_source

word_probs_by_source = calulate_conditionals(X, Y, [word for word in word_counts])
print(f"The probability of seeing the word 'wall' in Trumps speechs was {word_probs_by_source[source_IDs['trumpSpeech']]['wall']}")
print(f"The probability of seeing the word 'thou' in shakespeare was {word_probs_by_source[source_IDs['shakespeare']]['thou']}")
print(f"The probability of seeing the word 'wall' in chess was {word_probs_by_source[source_IDs['chess']]['wall']}")

 ## This is everything we need to learn from the data to make predictions!


In [ ]:
NB_model = calulate_conditionals(X, Y, [word for word in word_counts])

Lets check the output but showing the most likely words for each source

In [ ]:
for gen, source in data_files:
    word_probs = NB_model[source_IDs[source]].items()
    print(source)
    print(sorted(word_probs, key = lambda x: x[1])[-20:])

Now we can make predictions for single words! Lets write a function which takes a word and returns the most likely source.

In [ ]:
def get_word_probabilities(model, word):
    """ Returns a dict giving
    the ID of a source and the probability of word
    coming from that source.
    """

    # We want to fill source_probs
    # with the likelihood of a word coming from each source.
    # E.G [('Twitter', 0.9), ('Wikipedia', 0.5)]
    source_probs = {}
    for source in model:
      # Get probability of word:
      word_prob = model[source][word]
      
      # Add to list
      source_probs[source] = word_prob
    return source_probs

def predict_single_word(model, word):
    """ Return the source ID word is most likely
    to have come from.
    """
    source_probs = get_word_probabilities(model, word)

    # Sort the list
    # Max will be the last element
    # take the source
    most_likely_source = sorted(source_probs.items(), key=lambda x: x[1])[-1][0]
    
    return most_likely_source

        

# Lets Test it out! How good are our predictions?

Can you beat the computer?

In [ ]:
# Run this cell to initialize scores
word_pred_games_played = 0
word_pred_human_correct = 0
word_pred_ML_correct = 0

In [ ]:
# Run this cell to play the game!
s, label_opt1 = random.choice(data_files)
s, label_opt2 = random.choice(data_files)
source_gen, true_label = random.choice(data_files)
label_options = [label_opt1, label_opt2, true_label]
random.shuffle(label_options)
lines = source_gen()
line = random.choice(lines)
word = random.choice(line.split())
print("Word is", word)
predicted_label = input("Where did this word come from? {} or {} or {}".format(*label_options))
ML_predicted_label_ID = predict_single_word(NB_model, word)
ML_predicted_label = ID_to_source[ML_predicted_label_ID]
print("You were", "Right" if true_label == predicted_label else "Wrong", "!")
print("The computer was", "Right" if true_label == ML_predicted_label else "Wrong", "!")
print("Your predicted label (y^) was: {}. The computer predicted {}. The true label (y) was {}".format(predicted_label, ML_predicted_label, true_label))
word_pred_games_played += 1
word_pred_human_correct += (true_label == predicted_label)
word_pred_ML_correct += (true_label == ML_predicted_label)
print("Your score:", str(word_pred_human_correct / word_pred_games_played), "AI score:", str(word_pred_ML_correct / word_pred_games_played))

## Predicting Sentences

Now that we can classify the source of individual words, how can we upgrade to sentences?

Lets think about probability a little bit. What is the probability of a heads when a coin is flipped? 
$$p(H) = \frac{1}{2}$$
Now what is the probability of flipping two heads in a row?
$$ p(H) \times p(H) = \frac{1}{2} \times \frac{1}{2} = \frac{1}{4}$$

We can check this is correct, as there are four possible options, HH, HT, TH, TT, and all are equally likely.

Can we do something similar with a sentence? We have the probability of each word coming from each source, can we just multiply them all together to get the probability of the sentence coming from a source?



### Yes! (Kind of)

It turns out that yes, we can! If we make some assumptions...

If we assume that all words are independant from each other, this works! This is actually exactly how Naive Bayes works!
(Though this may not be so realistic...)

So, lets put all of our intuition together!
1. Firstly, we want to predict the probability of a label (wikipedia, twitter, etc,) given some sentence ("Follow me please!!!", "Auckland was founded...").
    1. In statistics, this can be written p(y=Y|x=X)
2. This is *hard* to calculate, so we use bayes rule to flip it around. We predict the probability of the sentence we have coming from each of the sources!
    1. This is now: p(x=X|y=Y)
3. The probability of the sentence coming from a source can be thought of as the product of each *word* coming from that source!
    1. p(Follow me please|Twitter) = p(Follow|Twitter) x p(me|Twitter) x p(please|Twitter)
4. Once probabilities are calculated, we can take the most likely label as our prediction!
    1. If p(Follow me please|Twitter) = 0.2 and p(Follow me please|Wikipedia) = 0.001, we can predict the sentence "Follow me please" is from Twitter!

In [ ]:
import math
def get_sentence_probs(model, sentence):
    """ Returns a list of tuples giving
    the ID of a source and the probability of sentence
    coming from that source.
    """
    source_probs = {}

    # Initialize the probabilities to be 1
    for source in model:
      source_probs[source] = 1

    for word in sentence.split():
        source_probabilities = get_word_probabilities(model, word)
        for source, prob in source_probabilities.items():
            source_probs[source] += math.log(prob)

    return source_probs.items()

def predict_sentence(model, sentence):
    source_probs = get_sentence_probs(model, sentence)
    return sorted(source_probs, key= lambda x: x[1])[-1][0]

## Game 2: Full Sentences!

In [ ]:
sent_pred_games_played = 0
sent_pred_human_correct = 0
sent_pred_ML_correct = 0

In [ ]:
import random
s, label_opt1 = random.choice(data_files)
s, label_opt2 = random.choice(data_files)
source_gen, true_label = random.choice(data_files)
label_options = [label_opt1, label_opt2, true_label]
random.shuffle(label_options)
lines = source_gen()
line = random.choice(lines)
print("Line is:", line)
predicted_label = input("Where did this word come from? {} or {} or {}".format(*label_options))
ML_predicted_label_ID = predict_sentence(NB_model, line)
ML_predicted_label = ID_to_source[ML_predicted_label_ID]
print("You were", "Right" if true_label == predicted_label else "Wrong", "!")
print("The computer was", "Right" if true_label == ML_predicted_label else "Wrong", "!")
print("Your predicted label (y^) was: {}. The computer predicted {}. The true label (y) was {}".format(predicted_label, ML_predicted_label, true_label))
sent_pred_games_played += 1
sent_pred_human_correct += (true_label == predicted_label)
sent_pred_ML_correct += (true_label == ML_predicted_label)
print("Your score:", str(sent_pred_human_correct / sent_pred_games_played), "AI score:", str(sent_pred_ML_correct / sent_pred_games_played))

## Evaluating

So, after playing the ML algorithm, it seems pretty good!
The next step is to test *how good*. Evaluation is an important part of machine learning, to test how good our models actually are. This is required for two things:
1. Making sure we can actually put it into critical decision making roles
2. Making sure we understand *what* its decisions are based on
3. Making sure any changes we make actually are improving it!

Lets look at the most basic form of evaluation, how well the predicted labels match the known labels. This is called *accuracy*!

In [ ]:
num_right = 0
num_wrong = 0
for x, y in observations:
    ML_predicted_label_ID = predict_sentence(NB_model, x)
    ML_correct = ML_predicted_label_ID == y
    num_right += ML_correct
    num_wrong += not ML_correct
accuracy = num_right / (num_right + num_wrong)
print(accuracy)

Lets compare to a library implementation!
Scikit-Learn is a very popular, simple to use ML library in python. Lets compare to their built in naive bayes model.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# SKlearn doesn't work directly on words like our implementation, but needs them to be
# vectorized first. This lets it be more general, and work on more than just text!
cv = CountVectorizer()
X_cv = cv.fit_transform(X)

# We are going to shuffle the sentences, and then take 90% to train on
# This will be closer to real training you'll see in examples
X_cv, X_shuffle, Y_shuffle = shuffle(X_cv, X, Y) 
train_len = int(len(X) * 0.9)
X_cv_train = X_cv[:train_len]
X_cv_test = X_cv[train_len:]
Y_train = Y_shuffle[:train_len]
Y_test = Y_shuffle[train_len:]
X_sentence_train = X_shuffle[:train_len]
X_sentence_test = X_shuffle[train_len:]

# Fit both models to the selected data
sklearn_model = MultinomialNB().fit(X_cv_train, Y_train)
our_model = calulate_conditionals(X_sentence_train, Y_train, [w for w in word_counts])

In [ ]:
our_right = 0
our_wrong = 0
sk_right = 0
sk_wrong = 0
num_to_test = 1000
wrong_examples = []

# For each example, we predict with both models and increment right if they
# get it right, otherwise wrong.
for x_cv, x, y in zip(X_cv_test[:num_to_test], X_sentence_test[:num_to_test], Y_test[:num_to_test]):
    if len(x.split()) < 1:
        continue
    ML_predicted_label_ID = predict_sentence(our_model, x)
    sk_predicted_label_ID = sklearn_model.predict(x_cv)[0]
    our_right += (ML_predicted_label_ID == y)
    our_wrong += (ML_predicted_label_ID != y)
    sk_right += (sk_predicted_label_ID == y)
    sk_wrong += (sk_predicted_label_ID != y)
    if ML_predicted_label_ID != y:
        wrong_examples.append((x, ID_to_source[y], ID_to_source[ML_predicted_label_ID]))
        
our_accuracy = our_right / (our_right + our_wrong)
sk_accuracy = sk_right / (sk_right + sk_wrong)
print("Our accuracy:", our_accuracy, "SK Learn accuracy:", sk_accuracy)

In [ ]:
# Show some examples of ones we got wrong
# Sentence, True label, predicted label
wrong_examples[:5]

# Conclusion

We've done it! We have build a ML model from scratch to classify where a sentence came from, with over 90% accuracy!

As we saw with the Scikit-Learn comparison, our model was pretty simple. There are many many more complex methods out there, its an exciting world!

Next, we will look at how we can use very similar ideas to generate *new* text!

# Text Generation

So far, we have looked at *discriminative* machine learning. This means our model learns patterns which allow it to *discriminate* (or, tell the difference between) different classes. 

Now, we will look at *generative* machine learning! Our model will learn patterns which allow it to generate *new* data!

### Basic Naive Bayes Model

Can we *generate* text with our NB model? Lets try just building a sentence by taking the most likely word.

In [ ]:
def NB_gen_sentence_1(model, source, num_words=10):
    """ Only predict most likely word
    """
    # Get a list of (word, likelihood) pairs for the source
    word_likelihoods = model[source].items()

    # sort the list, so the most likely words are at the start of the list (this is reverse of normal sorting)
    # The key tells sorted to look at the second element for sorting, the likelihood
    sorted_likelihoods = sorted(word_likelihoods, reverse=True, key=lambda x: x[1])

    sentence_list = []
    for i in range(num_words):
      likely_word, likelihood = sorted_likelihoods[0]
      sentence_list.append(likely_word)
    return ' '.join(sentence_list)
    
text_source = "shakespeare"
print(NB_gen_sentence_1(our_model, source_IDs[text_source]))

We seem to have an issue, if we just pick the most likely word, we get the same thing every time!

Lets try sampling different words based on the probability, rather than just taking the top one.

In [ ]:

def NB_gen_sentence_2(model, source, num_words, num_top_word):
    """Predict a random likely word
    """
    # Get a list of (word, likelihood) pairs for the source
    word_likelihoods = model[source].items()

    # sort the list, so the most likely words are at the start of the list (this is reverse of normal sorting)
    # The key tells sorted to look at the second element for sorting, the likelihood
    sorted_likelihoods = sorted(word_likelihoods, reverse=True, key=lambda x: x[1])

    sentence_list = []
    for i in range(num_words):
      # Select the top words
      top_words = sorted_likelihoods[:num_top_word]

      selected_word, selected_likelihood = random.choice(top_words)
      sentence_list.append(selected_word)
    return ' '.join(sentence_list)
    
text_source = "shakespeare"
print(NB_gen_sentence_2(our_model, source_IDs[text_source], 10, 1000))

This seems to work! But has a major issue.

Our Naive Bayes model only captures patterns to do with what words are in a sentence, but not their order. This is enough for a *discriminative* model, but not a *generative* model. We really want patterns which capture relationships between words, and the position of words in a sentence too. (These might also be helpful for disciminative models too!)

Lets think about how we might capture these patterns in our data.

### N-Grams

Up until now, we have only looked at words individually. This is called a *bag-of-words* model. But really, a sentence is a *sequence* of words. The next word in a sentence depends on the rest of the words, not only on the source of the text. 

N-grams model how the next word in a sentence depends on *previous* words. The *N* refers to how far back we look. Lets start with bi-grams (bi = 2, tri = 3, etc). This means that we need to capture the probability of a word, given the source, given the previous word. Our probability formula for word $w_i$ looks like:
$$ p(w_i | w_{i-1}, source)$$.

For example, before our probability of the words 'and' and 'the' were pretty high in wikipedia. $p(and|Wikipedia) = high$, $p(the|Wikipedia) = high$. This means that in our generated sentences, the sequence 'the and' or 'and the' are pretty common! But we know that while 'and the' is actually pretty common in real sentences, 'the and' doesn't make much sence. By including the previous word in our probabilities, we can account for this! $$p(the | and, Wikipedia) = High, p(and | the, Wikipedia) = Low$$

So how do we learn this new bi-gram model? Well, we only need to extend our code a little. Rather than counting how many times 'the' appears in Wikipedia as we did to calculate $p(the|Wikipedia)$, we need to include the previous word too. So we need to count, for example, how many times 'the' appeared after 'and' in wikipedia for $p(the|and,Wikipedia)$. We need to count this for every possible word it could appear after, how many times it appeared after $w_{i-1}$, $p(the|w_{i-1},Wikipedia)$

Lets update our NB model with bi-grams!

Lets start with extracting the ngrams from a sentence:

In [ ]:
def extract_ngrams(sentence, N):
  """ Sentence is the input string
  N is the number of words to consider at one.
  So if N = 2, we consider the current word and 1 previous word.
  """
  ngrams = []
  sentence_words = sentence.split()
  current_ngram = ['' for i in range(N)]
  for word in sentence_words:
    # Remove the word at the front of the ngram,
    # and add the current word to the end.
    current_ngram = (*current_ngram[1:], word)
    previous_words = current_ngram[:-1]
    ngrams.append((previous_words, word))
  return ngrams

bigrams = extract_ngrams("Hello, this is a sentence about bigrams", 2)
trigrams = extract_ngrams("Hello, this is a sentence about trigrams", 3)
print(bigrams[:10])
print(trigrams[:10])

Previously, we stored the count of each word for each source label. Now, we need to store the count of each word for each source label AND set of previous words.

We can consider this as a new level in the probability dictionary. First, we look up the source label to get a map of (previous words) -> following word. Then we look up the set of previous words in our sentence, to find possible following words.

In [ ]:
  def get_ngram_counts(sentences, N):
    # fill ngram_counts so it contains the number of times each word occurs for each set of previous words.
    # word_counts = {
    #       ('and',): {
    #           'the': 15,
    #           'then': 7,
    #        },
    #       ('founded',): {
    #           'Auckland', 
    #       },
    #       ....
    #}
    ngram_counts = {}

    # We now need to store a total for each set of previous words.
    total_num_ngrams = {}

    # We want to look though all the training sentences,
    for sentence in sentences:
      ngrams = extract_ngrams(sentence, N)
      for previous_words, word in ngrams:

        # Initialize the counts for the ngram if we haven't seen it yet
        if previous_words not in ngram_counts:
          ngram_counts[previous_words] = {}
          total_num_ngrams[previous_words] = 0

        # Increment the total
        total_num_ngrams[previous_words] += 1

        # Add word to dictionary if not already in it
        if word not in ngram_counts[previous_words]:
          ngram_counts[previous_words][word] = 0
      
        # Add to the count for the word
        ngram_counts[previous_words][word] += 1
    return ngram_counts, total_num_ngrams
  
  # Inspect our ngram counts!
  test_sentences = ["Hello, this is a sentence about bigrams", "Hello, this is a sentence about trigrams"]
  test_counts, test_totals = get_ngram_counts(test_sentences, 2)
  print(test_counts)
  for previous_word in test_counts:
    print(f"If the previous word is {previous_word}, the possible next words were: {test_counts[previous_word]}")

In [ ]:
def get_probabilities_from_ngrams(ngram_counts, total_num_ngrams):
  ngram_probabilities = {}

  for previous_words in ngram_counts:
    ngram_probabilities[previous_words] = {}

    # Get the total of times this set of previous words was seen
    ngram_total = total_num_ngrams[previous_words]

    for following_word in ngram_counts[previous_words]:

      # Get the number of times the word appeard after the set of previous words
      w_count = ngram_counts[previous_words][following_word]

      # Calc probability, and save to the dictionary
      ngram_probabilities[previous_words][following_word] = w_count / ngram_total
  return ngram_probabilities

# Inspect our bigrams!
bigram_counts, total_bigram_words = get_ngram_counts(X, 2)
word_probabilities = get_probabilities_from_ngrams(bigram_counts, total_bigram_words)
for previous_word in list(word_probabilities.keys())[:5]:
  print(f"If the previous word is {previous_word}, the possible next words were: {word_probabilities[previous_word]}")

In [ ]:
  def calulate_conditionals_Ngram(X, Y, N):
    # fill word_probs_by_source so it contains a dict for each source
    # each containing each word and probability only in that source
    # word_probs_by_source = {
    #       'Twitter': {
    #              'founded': 0.001,
    #               ...,
    #        },
    #       'Wikipedia': {
    #              'founded': 0.05,
    #              ...,
    #       }
    #}
    
    # Get a list of the possible sources
    # (Note: this is an ID not a name)
    sources = set(Y)
    ngram_probs_by_source = {}

    for s in sources:
      # Selects all the training examples which come from this source
      source_training_data = [x for x,y in zip(X, Y) if y == s]

      # Extract the counts and total for the specific source
      source_counts, source_total = get_ngram_counts(source_training_data, N)

      # Transform into probabilities
      source_probabilities = get_probabilities_from_ngrams(source_counts, source_total)

      #Save to the dictionary
      word_probs_by_source[s] = source_probabilities
    return word_probs_by_source

bigram_model = calulate_conditionals_Ngram(X, Y, 2)

Lets test our bigrams! The following cell lets you pick a word and prints the 5 most likely words to follow it.

In [ ]:

source_label = "shakespeare"
bigram = bigram_model[source_IDs[source_label]]
first_word = input("Select first word")
sorted_bigrams = sorted(bigram[(first_word,)].items(), key=lambda x: x[1])
print("Most likely next words:")
print(sorted_bigrams[-5:])

Now lets use bigrams to generate our text! Instead of sampling from all words from a source, we take into account the previously generated word!

In [ ]:
def NB_gen_sentence_3(model, source, num_words, num_top_word, N):
    """Predict a random likely word
    """
    # Select the ngrams for this source
    ngrams_for_source = model[source]

    # Setup the Ngram for the current sentence we are generating
    # Remember for an Ngram of N words, we have N-1 previous words
    # and the word we are generating is the remaining one
    current_ngram = [''] * N

    sentence_list = []
    for i in range(num_words):
      # The previous words are the previous ngram, without the first word (now too old)
      previous_words = current_ngram[1:]

      # Get a list of (word, likelihood) pairs for the current set of previous words
      word_likelihoods = ngrams_for_source[tuple(previous_words)].items()

      # sort the list, so the most likely words are at the start of the list (this is reverse of normal sorting)
      # The key tells sorted to look at the second element for sorting, the likelihood
      sorted_likelihoods = sorted(word_likelihoods, reverse=True, key=lambda x: x[1])

    
      # Select the top words
      top_words = sorted_likelihoods[:num_top_word]

      selected_word, selected_likelihood = random.choice(top_words)
      sentence_list.append(selected_word)

      # Update the current ngram
      current_ngram = [*previous_words, selected_word]
    return ' '.join(sentence_list)
    
text_source = "shakespeare"
print(NB_gen_sentence_3(bigram_model, source_IDs[text_source], 10, 1000, 2))

We run into an issue again, where we may not have seen a particular ngram before. How can we predict the next word?


In [ ]:
def NB_gen_sentence_3(model, source, num_words, num_top_word, N):
    """Predict a random likely word
    """
    # Select the ngrams for this source
    ngrams_for_source = model[source]

    # Setup the Ngram for the current sentence we are generating
    # Remember for an Ngram of N words, we have N-1 previous words
    # and the word we are generating is the remaining one
    current_ngram = [''] * N

    sentence_list = []
    for i in range(num_words):
      # The previous words are the previous ngram, without the first word (now too old)
      previous_words = current_ngram[1:]

      # Check previous_words was seen in the data
      # If not, just restart the ngram, like a new sentence.
      if tuple(previous_words) not in ngrams_for_source:
        current_ngram = [''] * N
        previous_words = current_ngram[1:]
      # Get a list of (word, likelihood) pairs for the current set of previous words
      word_likelihoods = ngrams_for_source[tuple(previous_words)].items()

      # sort the list, so the most likely words are at the start of the list (this is reverse of normal sorting)
      # The key tells sorted to look at the second element for sorting, the likelihood
      sorted_likelihoods = sorted(word_likelihoods, reverse=True, key=lambda x: x[1])

    
      # Select the top words
      top_words = sorted_likelihoods[:num_top_word]

      selected_word, selected_likelihood = random.choice(top_words)
      sentence_list.append(selected_word)

      # Update the current ngram
      current_ngram = [*previous_words, selected_word]
    return ' '.join(sentence_list)
    
text_source = "shakespeare"
print(NB_gen_sentence_3(bigram_model, source_IDs[text_source], 10, 10, 2))

We get more coherent sentences! But we can see that the patterns are not very long, focus tends to switch quickly. This is because our patterns only have a "memory" of one word, we can only make decisions one word back. We can increase this by using longer N-grams, lets try a tri-gram!

In [ ]:
trigram_model = calulate_conditionals_Ngram(X, Y, 3)

In [ ]:
text_source = "trumpSpeech"
print(NB_gen_sentence_3(trigram_model, source_IDs[text_source], 50, 10, 3))

But the women of suburbia don’t like me. I think they want anything. I don’t think that’s appropriate. When those votes are corrupt, when they’re irregular, when they get to eat very well. So what happened is they made this into a massive event and people are leaving. But if


We now see much more coherent sentences, even more so than the bigram model! Can we increase the N-gram length even further?

Give it a try! However, we start to run into an issue. The more we increase the length, the less data we have for each N-gram. For example, if we looked for patterns 10 words long, it is likely we only would get the exact sentences that appear in the training data. In other words, our patterns are *too* detailed, so have fit too closely to the training data. 

This is very very important problem in machine learning! We are interested in patterns which *generalize* to new data. If we look too deeply into our data, we can always find more and more complex patterns, but are they actually useful?

# Conclusion

In this talk, we have focused on simple models for one small area machine learning can be applied to. 

I encorage you to take what you have learned, and think up some more areas where classification and generation can be applied!